In [ ]:
from transformers import pipeline
import gradio as gr

# Initialize the text generation pipeline with a model
generator = pipeline('text-generation', model='gpt2-medium')

def generate_story(prompt, max_length=200):
    """
    Generate a story from a given prompt using the text generation model.

    Args:
        prompt (str): The starting text or idea for the story
        max_length (int): Maximum length of the generated story

    Returns:
        str: Generated story text
    """
    # Generate text with the model
    generated = generator(
        prompt,
        max_length=max_length,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.8
    )

    # Extract and clean the generated text
    story = generated[0]['generated_text']
    return story.strip()

# Gradio interface function
def story_interface(prompt, max_words):
    """
    Interface function for Gradio to handle user input and display output.

    Args:
        prompt (str): User input prompt
        max_words (int): Maximum number of words for the story

    Returns:
        str: Generated story
    """
    if not prompt:
        return "Please enter a prompt to generate a story."

    try:
        max_words = int(max_words)
        if max_words < 50 or max_words > 500:
            return "Please choose a word count between 50 and 500."

        story = generate_story(prompt, max_length=max_words)
        return story
    except ValueError:
        return "Please enter a valid number for maximum words."

# Create Gradio interface
iface = gr.Interface(
    fn=story_interface,
    inputs=[
        gr.Textbox(lines=2, placeholder="Enter your story prompt here...", label="Story Prompt"),
        gr.Slider(minimum=50, maximum=500, value=200, step=10, label="Max Words")
    ],
    outputs=gr.Textbox(label="Generated Story"),
    title="Text to Story Generator",
    description="Enter a prompt to generate a short story using AI. Adjust the maximum word count as needed."
)

# Launch the interface
if __name__ == "__main__":
    iface.launch()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1384499cc2739d39de.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import torch
from diffusers import CogVideoXPipeline
from moviepy.editor import ImageSequenceClip
import numpy as np
import os
from IPython.display import Video

def generate_video(prompt="Five little monkeys jumping on a bed", negative_prompt="low quality, blurry, distorted", num_inference_steps=50, guidance_scale=6.0, output_path="/content/output_video.mp4"):
    try:
        # Initialize the CogVideoX text-to-video pipeline
        pipe = CogVideoXPipeline.from_pretrained(
            "THUDM/CogVideoX-2b",
            torch_dtype=torch.float16
        )

        # Enable memory optimizations
        pipe.enable_model_cpu_offload()
        pipe.vae.enable_tiling()
        pipe.vae.enable_slicing()

        # Set device to CUDA if available, else CPU
        device = "cuda" if torch.cuda.is_available() else "cpu"
        pipe = pipe.to(device)

        # Set parameters for a 5-second video (8 FPS * 5 seconds = 40 frames)
        num_frames = 40
        fps = 8

        # Generate video frames
        result = pipe(
            prompt=prompt,
            negative_prompt=negative_prompt,
            num_frames=num_frames,
            num_inference_steps=num_inference_steps,
            guidance_scale=guidance_scale,
            generator=torch.Generator(device=device).manual_seed(42)
        )
        frames = result.frames  # CogVideoX returns frames directly

        # Convert frames to numpy arrays for MoviePy
        frames_np = [np.array(frame) for frame in frames]

        # Create MP4 video using MoviePy
        clip = ImageSequenceClip(frames_np, fps=fps)
        clip.write_videofile(output_path, codec="libx264", audio=False)
        clip.close()

        # Clean up to free memory
        del pipe
        torch.cuda.empty_cache()

        print(f"Video saved to {output_path}")
        return output_path
    except Exception as e:
        print(f"Error: {str(e)}")
        return None

# Colab-specific execution
if __name__ == "__main__":
    # Define parameters
    prompt = "Five little monkeys jumping on a bed"
    negative_prompt = "low quality, blurry, distorted"
    num_inference_steps = 50
    guidance_scale = 6.0
    output_path = "/content/output_video.mp4"

    # Generate the video
    video_path = generate_video(
        prompt=prompt,
        negative_prompt=negative_prompt,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale,
        output_path=output_path
    )

    # Display the video in Colab if generated successfully
    if video_path and os.path.exists(video_path):
        display(Video(video_path, embed=True))

model_index.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

Fetching 14 files:   0%|          | 0/14 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.53G [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

scheduler_config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/879 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.39G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]